In [1]:
import os

from prism_pruner.utils import read_xyz

os.chdir("/home/Coding")

ensemble = read_xyz("t1.xyz")
ensemble.atomcoords.shape

(1, 33, 3)

In [2]:
from prism_pruner.graph_manipulations import graphize
from prism_pruner.torsion_module import _get_rotation_mask, rotate_dihedral
from prism_pruner.utils import write_xyz

graph = graphize(ensemble.atomcoords[0], ensemble.atomnos)
graph.add_edge(32, 4)

In [4]:
from copy import deepcopy

import numpy as np

from prism_pruner.utils import align_structures

structs = []
for angle in (120, 240):
    for dihedral in (
        (2, 0, 4, 14),
        (32, 27, 25, 23),
    ):
        mask = _get_rotation_mask(graph, dihedral)
        newcoords = deepcopy(ensemble.atomcoords[0])
        rotate_dihedral(newcoords, dihedral, angle, mask)
        structs.append(newcoords)


with open("bimol_ens.xyz", "w") as f:
    for c in align_structures(np.array(structs), np.array([4, 14, 15])):
        write_xyz(c, ensemble.atomnos, f)

In [5]:
from prism_pruner.rmsd import rmsd_and_max_numba

for tgt in structs[1:]:
    rmsd, maxdev = rmsd_and_max_numba(structs[0], tgt, center=True)
    print(f"rmsd: {rmsd:.2f}, maxdev: {maxdev:.2f}")

rmsd: 2.06, maxdev: 3.62
rmsd: 1.78, maxdev: 3.20
rmsd: 2.68, maxdev: 4.39


In [6]:
from prism_pruner.torsion_module import (
    _get_hydrogen_bonds,
    _get_torsions,
    _is_nondummy,
    get_angles,
    rotationally_corrected_rmsd_and_max,
)
from prism_pruner.utils import get_double_bonds_indices

graph.remove_edge(32, 4)

hbs = _get_hydrogen_bonds(structs[0], ensemble.atomnos, graph)

torsions = _get_torsions(
    graph,
    hydrogen_bonds=hbs,
    double_bonds=get_double_bonds_indices(structs[0], ensemble.atomnos),
    keepdummy=True,
    mode="symmetry",
)

# only keep dummy rotations (checking both directions)
torsions = [
    t
    for t in torsions
    if not (_is_nondummy(t.i2, t.i3, graph) and (_is_nondummy(t.i3, t.i2, graph)))
]
# print(f"1: {torsions}")
# since we only compute RMSD based on heavy atoms, discard
# quadruplets that involve hydrogen atoms
torsions = [
    t
    for t in torsions
    if (1 not in [ensemble.atomnos[i] for i in t.torsion])
    or (
        1 in [ensemble.atomnos[i] for i in t.torsion]
        and (t.torsion[0] in hbs[0] or t.torsion[3] in hbs[0])
    )
]
# print(f"2: {torsions}")
# get torsions angles
angles = [get_angles(t, graph) for t in torsions]

# Used specific directionality of torsions so that we always
# rotate the dummy portion (the one attached to the last index)
torsions_ids = [
    list(t.torsion) if _is_nondummy(t.i2, t.i3, graph) else list(reversed(t.torsion))
    for t in torsions
]

for tgt in structs[1:]:
    rmsd, maxdev = rotationally_corrected_rmsd_and_max(
        structs[0],
        tgt,
        ensemble.atomnos,
        torsions_ids,
        graph,
        angles,
        #    debugfunction=print,
    )
    print(f"rmsd: {rmsd:.2f}, maxdev: {maxdev:.2f}")

rmsd: 0.03, maxdev: 0.03
rmsd: 0.03, maxdev: 0.03
rmsd: 0.03, maxdev: 0.03


In [7]:
hbs

[[4, 32]]

In [8]:
from prism_pruner import prune_by_rmsd_rot_corr

c, _ = prune_by_rmsd_rot_corr(
    structs,
    ensemble.atomnos,
    graph,
    logfunction=print,
)

c.shape


 >> Dihedrals considered for rotamer corrections:
 1  - [14, 4, 0, 1]         : COCC : 3-fold
 2  - [32, 27, 25, 21]      : HOCC : 3-fold




(1, 33, 3)